In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# save files to "test_titanic" folder
train_org = pd.read_csv("test_titanic/train.tsv", delimiter='\t')
test_org = pd.read_csv("test_titanic/test.tsv", delimiter='\t')

In [20]:
def drop_attribute(data):
    data = data.drop('id', axis=1)
    data = data.drop('embarked', axis=1)
    return data

In [24]:
def expand_attribute(data):
    data = pd.get_dummies(data)
    return data

In [31]:
from sklearn.preprocessing import Imputer
def replace_nan2median(data):
    imp = Imputer(missing_values='NaN', strategy='median', axis=0, verbose=0, copy=True)
    imp.fit(data)
    data = imp.transform(data)
    return pd.DataFrame(data)

In [33]:
# preprocessing training data
trainY = train_org.loc[:,"survived"]
trainX = train_org.drop(columns='survived')
trainX = drop_attribute(trainX)
trainX = expand_attribute(trainX)
trainX = replace_nan2median(trainX)

In [35]:
# preprocessing test data
testX = drop_attribute(test_org)
testX = expand_attribute(testX)
testX = replace_nan2median(testX)

スケーリング

In [39]:
from sklearn.preprocessing import StandardScaler
def scale(data):
    ss = StandardScaler()
    data_std = ss.fit_transform(data)
    return pd.DataFrame(data_std)

In [41]:
trainX = scale(trainX)
testX = scale(testX)

学習(pipeline)

In [42]:
from sklearn.pipeline import Pipeline
def run_pipeline(estimators):
    pipe = Pipeline(estimators)
    return pipe

In [44]:
from sklearn.model_selection import GridSearchCV
def run_GridSearch(pipe, param):
    gs = GridSearchCV(pipe, param, n_jobs=1, verbose=2)
    return gs

In [53]:
# 前処理に変更を加えない場合は、以下のパラメータ・識別器の変更をおこない検証する。精度が悪い場合は前処理を見直すこと。
from sklearn.svm import SVC

C_range = [1e-3, 1e-2, 1, 1e2, 1e5]
estimators = [('clf', SVC())]
param = {'clf__C':C_range,
         'clf__kernel': ['linear', 'rbf']}

gs = run_GridSearch(run_pipeline(estimators), param)
gs.fit(np.array(trainX), np.array(trainY))
gs.best_params_, gs.best_score_, gs.best_estimator_

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] clf__C=0.001, clf__kernel=linear ................................
[CV] ................. clf__C=0.001, clf__kernel=linear, total=   0.0s
[CV] clf__C=0.001, clf__kernel=linear ................................
[CV] ................. clf__C=0.001, clf__kernel=linear, total=   0.0s
[CV] clf__C=0.001, clf__kernel=linear ................................
[CV] ................. clf__C=0.001, clf__kernel=linear, total=   0.0s
[CV] clf__C=0.001, clf__kernel=rbf ...................................
[CV] .................... clf__C=0.001, clf__kernel=rbf, total=   0.0s
[CV] clf__C=0.001, clf__kernel=rbf ...................................
[CV] .................... clf__C=0.001, clf__kernel=rbf, total=   0.0s
[CV] clf__C=0.001, clf__kernel=rbf ...................................
[CV] .................... clf__C=0.001, clf__kernel=rbf, total=   0.0s
[CV] clf__C=0.01, clf__kernel=linear .................................
[CV] ...........

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV] ................. clf__C=100.0, clf__kernel=linear, total=   0.1s
[CV] clf__C=100.0, clf__kernel=linear ................................
[CV] ................. clf__C=100.0, clf__kernel=linear, total=   0.0s
[CV] clf__C=100.0, clf__kernel=linear ................................
[CV] ................. clf__C=100.0, clf__kernel=linear, total=   0.0s
[CV] clf__C=100.0, clf__kernel=rbf ...................................
[CV] .................... clf__C=100.0, clf__kernel=rbf, total=   0.0s
[CV] clf__C=100.0, clf__kernel=rbf ...................................
[CV] .................... clf__C=100.0, clf__kernel=rbf, total=   0.0s
[CV] clf__C=100.0, clf__kernel=rbf ...................................
[CV] .................... clf__C=100.0, clf__kernel=rbf, total=   0.0s
[CV] clf__C=100000.0, clf__kernel=linear .............................
[CV] .............. clf__C=100000.0, clf__kernel=linear, total= 1.0min
[CV] clf__C=100000.0, clf__kernel=linear .............................
[CV] .

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:  3.9min finished


({'clf__C': 1, 'clf__kernel': 'rbf'}, 0.8224719101123595, Pipeline(memory=None,
      steps=[('clf', SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
   decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
   max_iter=-1, probability=False, random_state=None, shrinking=True,
   tol=0.001, verbose=False))]))

'clf__C': 1, 'clf__kernel': 'rbf'　に従い次工程へ移る

In [59]:
testY_pred = gs.predict(testX)

In [66]:
submit = pd.DataFrame()
submit[0] = test_org.iloc[:,0] # insert id to variable
submit[1] = testY_pred # insert predict to variable

tsvへ出力する

In [69]:
submit.to_csv("test_titanic/second.tsv", sep='\t',  header=False, index=False)

In [131]:
sample.to_csv("first.tsv", sep='\t', header=False, index=False)

https://signate.jp/competitions/102/leaderboard